### 자소서 생성기


1. 기업 분석 및 직무 분석 (되게 자잘한 것까지)
2. 내 경험 모듈, (1) 내용 바탕으로 경험 모듈 가장 적합할만한 경험 n개 뽑기
3. (2)의 내용과 자소서 문항 및 constraint 고려해서 초안 작성
---> 이때 모든 자소서 문항 한번에 작성되게 만들까 따로따로 만들게 할까
4. 내가 강조하고 싶은 부분을 피드백으로 받고 constraint에 맞게 (3) 내용 재작성

In [1]:
from openai import OpenAI
import os
import dotenv
import glob

In [2]:
## dotenv load
dotenv.load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

In [3]:
### RAG 시키기
# 1) 벡터스토어 생성
vs = client.vector_stores.create(name="how-to-get-a-job")

# 2) 파일 업로드 + 벡터스토어에 넣고(자동 청킹/임베딩) 완료까지 폴링
file_paths = glob.glob("./docs/**", recursive=True)
file_paths = [p for p in file_paths if os.path.isfile(p)]

file_streams = [open(p, "rb") for p in file_paths]
client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vs.id,
    files=file_streams,
)

VectorStoreFileBatch(id='vs_696632723f248191944e97b8c7a4c7ac', created_at=1768305266, file_counts=FileCounts(cancelled=0, completed=35, failed=0, in_progress=0, total=35), object='vector_store', status='in_progress', vector_store_id=None, name='how-to-get-a-job', description=None, usage_bytes=0, expires_after=None, expires_at=None, last_active_at=1768305272, metadata={})

In [4]:
conversation = client.conversations.create(
    metadata={"topic": "writing-cover-letter"}
)  # :contentReference[oaicite:1]{index=1}


In [5]:
def company_persona_department_correctness(company_name: str, department_name: str):
    question = f"""
{company_name}의 {department_name} 직무를 지원할 거야 아래 조건에 맞춰 정리해줘.
1. {company_name}의 인재상과 경영이념 (업로드 자료(file_search)에서만 근거로 사용)
2. {department_name} 직무에서 가장 중요한 능력과 요구사항 (반드시 web_search로 확인)
3. 현재 {company_name}의 경영 방향성과 임원들의 {department_name} 관련 방향성 (반드시 web_search로 확인, 근거/출처 포함)

규칙:
- 2, 3은 web_search를 수행하고, 출처를 포함해라. 검색으로 확인 안 되면 '확인 불가'라고 써라.
- 1은 업로드 자료에서 확인 안 되면 '업로드 자료에서 확인 불가'라고 써라.
- 아래 output 형식만 출력해라.

output:
1. (내용)
2. (내용)
3. (내용)
"""

    response = client.responses.create(
        model="gpt-5-mini-2025-08-07",
        reasoning={"effort": "medium"},
        input=[
            {
                "role": "system",
                "content": (
                    "You are an expert finance job application coach. "
                    "Write in Korean unless the user asks otherwise."
                ),
            },
            {"role": "user", "content": question},
        ],
        tools=[
            {"type": "file_search", "vector_store_ids": [vs.id]},
            {"type": "web_search"},
        ],
        include=[
            "file_search_call.results",
            "web_search_call.action.sources",
        ],
        conversation=conversation.id,  # 없으면 빼도 됨
    )

    return response.output_text

In [6]:
company_persona_response=company_persona_department_correctness(company_name='미래에셋 증권', department_name='파생상품 트레이딩')

In [7]:
def select_individual_experience(company_persona_response: str, n_experiences: int):
    question = f"""
아래 [이전 분석]을 참고해서, 업로드된 '경험 모듈' 문서(file_search)에서만
{n_experiences}개의 경험을 선택해줘.

[이전 분석]
{company_persona_response}

규칙(매우 중요):
- 경험은 반드시 '경험 모듈'에서 찾은 "하나의 경험" 단위로만 선택한다. (경험 섞기 금지)
- 총 {n_experiences}개를 정확히 출력한다.
- 각 번호는 서로 다른 경험이어야 한다.
- 경험 모듈에서 근거를 찾지 못하면 해당 경험은 선택하지 말고, 대신 "경험 모듈에서 찾을 수 없음"이라고 표기한다.
- 출력은 아래 형식만 준수한다(추가 텍스트 금지).

output:
1. 경험명/ID: (경험을 식별할 수 있는 고유 키: 경험명 또는 경험ID)
   - 자세한 내용: (상황/역할/행동/성과를 구체적으로)
   - 회사: (회사/조직명)
   - 지원 직무 연관성: (직무 요구역량과 1:1로 연결)
2. 경험명/ID: ...
...
{n_experiences}. 경험명/ID: ...
"""

    response = client.responses.create(
        model="gpt-5-mini-2025-08-07",
        reasoning={"effort": "medium"},
        input=[
            {
                "role": "system",
                "content": (
                    "You are an expert finance job application coach. "
                    "You MUST select experiences only from the uploaded 'experience module' documents via file_search. "
                    "Write in Korean unless the user asks otherwise."
                ),
            },
            {"role": "user", "content": question},
        ],
        tools=[
            {
                "type": "file_search",
                "vector_store_ids": [vs.id],
                # "max_num_results": 10,  # 필요하면 올리기
            }
        ],
        include=["file_search_call.results"],
        conversation=conversation.id,  # 멀티턴 유지 원치 않으면 제거 가능
    )

    return response.output_text


In [8]:
selection_experience_reponse=select_individual_experience(company_persona_response=company_persona_response, n_experiences=4)

In [9]:
selection_experience_reponse

'output:\n1. 경험명/ID: 이정회계법인 인턴_LSMC_2021  \n   - 자세한 내용: 회계법인 인턴으로 엑셀 기반 평가엔진을 Python으로 대체하는 과제 수행. BDT 모형 기반 전환사채 모델을 개발하고, 남은 기간에 LSMC(Least‑Squares Monte Carlo) 기반 조건부옵션 엔진을 논문과 기존 코드 분석을 통해 단계별로 구현함. 촉박한 일정 속에서 논문을 읽고 코드를 수정·보완하며 기한 내 모델을 완성했고, 이후에도 피드백을 반영하며 추가 업무 제안을 받음.    \n   - 회사: 이정회계법인  \n   - 지원 직무 연관성: 계량모델 구현·검증 역량 → 옵션 프라이싱·몬테카를로 모델링(정량역량), Python 기반 구현 능력(프로그래밍), 기한 내 결과물 전달(실행력/스트레스 관리)\n\n2. 경험명/ID: KRX_증권_파생상품_경시대회_2024  \n   - 자세한 내용: KRX 주최 파생상품 경시대회 참가 및 발표 경험. 데이터·시계열 분석을 통해 파생상품 관련 지표(VKOSPI 등)를 활용한 분석을 수행했고, 발표 중 회귀분석 변수 구성 및 시각화 관련 피드백을 수용해 결과물을 보완함.   \n   - 회사: KRX(대회 주최)  \n   - 지원 직무 연관성: 파생상품 시장·지표 이해(금융지식), 데이터 분석 및 시각화 역량(분석력), 피드백 수용 후 개선하는 협업능력(팀워크·커뮤니케이션)\n\n3. 경험명/ID: 월드퀀트_IQC_참가_및_수상  \n   - 자세한 내용: 월드퀀트(대회) IQC 참가하여 2위 입상(대회 성과 기록). 퀀트 문제 해결 과정에서 모델링·알고리즘 설계 및 성과 도출을 경험함.   \n   - 회사: WorldQuant IQC (대회)  \n   - 지원 직무 연관성: 계량적 문제 해결 능력(정량역량), 알고리즘/전략 설계 경험(전산·퀀트 역량), 성과 기반 결과 도출 능력(성과지향성)\n\n4. 경험명/ID: 코인_자동매매_페어트레이딩_프로젝트  \n   - 자세한 내용: 개인 프로젝트로

In [ ]:
selection_experience_reponse

In [17]:
def writing_prototype(company_persona_response, selection_experience_reponse):
    question = f"""
아래 [기업분석]과 [선택된 경험]을 참고해서, 업로드된 '자소서 문항' 문서(file_search)에서
해당 기업의 자소서 문항을 찾아 보고 초안을 작성해.

[기업분석]
{company_persona_response}

[선택된 경험]
{selection_experience_reponse}

규칙(매우 중요):
- 경험에 대한 고차원적인 고민을 진행해서 가치를 발굴해.
- 진짜 자기소개서 쓰듯이 이상한 기호나 말투 사용하지 마.
- 모든 경험을 다 적지말고, 몇몇 경험을 선택해서 깊은 이해를 진행해.

output:
1. (문항 그대로 작성)
(내용)

2. (문항 그대로 작성)
(내용)
...
"""

    response = client.responses.create(
        model="gpt-5-mini-2025-08-07",
        reasoning={"effort": "medium"},
        input=[
            {
                "role": "system",
                "content": (
                    "You are an expert finance job application coach. "
                    "You must use file_search to retrieve the company's official 자기소개서 문항 and character limits "
                    "from the uploaded documents, and you must obey those limits (including spaces). "
                    "Write as real self introduction paragraphs not as lists\n"
                    "Do not use emojis or emoticons\n"
                    "Do not use decorative or bullet symbols such as arrows or middle dots\n"
                    "Examples of forbidden symbols include → ← ⇒ ⇨ • · ▶ ※ ★ ☆ ✅ ❌ 🔥 😊\n"
                    "Normal sentence punctuation is allowed\n"
                    "Write in Korean."
                ),
            },
            {
                "role": "developer",
                "content": (
                    "Hard rules:\n"
                    "- Use ONLY the experiences provided in [선택된 경험]. Do NOT invent new experiences.\n"
                    "- Every selected experience must be used at least once across the whole application.\n"
                    "- For each prompt, do NOT exceed the specified character limit (including spaces). If the limit is unknown, "
                    "state '글자수 제한: 문서에서 확인 불가' and write a concise answer under 600 characters.\n"
                    "- Output format only. No extra commentary.\n"
                    "- Structure:\n"
                    "  1. <문항 원문>\n"
                    "  <내용>\n"
                    "  2. <문항 원문>\n"
                    "  <내용>\n"
                ),
            },
            {"role": "user", "content": question},
        ],
        tools=[
            {"type": "file_search", "vector_store_ids": [vs.id]}
        ],
        include=["file_search_call.results"],
        conversation=conversation.id,
    )

    return response.output_text


In [18]:
prototype_response = writing_prototype(company_persona_response=company_persona_response, selection_experience_reponse=selection_experience_reponse)

In [16]:
prototype_response

'1. Q1.본인이 생각하는 금융의 역할은 무엇인지, 그리고 미래에셋증권에 지원하게 된 동기를 구체적으로 기술해주세요. (500자)\n금융은 자본을 효율적으로 배분하고 리스크를 관리하여 실물과 투자자에게 안정적 가치를 제공하는 역할이라 생각합니다. 월드퀀트 IQC에서 2위로 입상하며 계량적 모델로 시장의 비효율을 찾고 해결하는 과정에서 금융의 실용적 의미를 체감했습니다. 미래에셋의 글로벌 플랫폼과 팀 어프로치, 장기적 투자원칙은 제 계량·협업 역량을 발휘해 고객 성공에 기여할 수 있는 최적의 무대라 판단해 지원했습니다.\n\n2. Q2.희망하는 직무와 관련하여 본인만의 차별화된 경쟁력은 무엇인지, 미래에셋증권에 입사하기 위해 어떤 준비를 하였는지 구체적으로 기술해 주세요.  (1000자)\n희망 직무는 파생상품 트레이딩(퀀트·헤지)입니다. 이정회계법인 인턴으로 엑셀 기반 평가엔진을 Python으로 재구현하고 BDT 기반 전환사채 모델을 개발한 뒤, 남은 기간에 LSMC(Least‑Squares Monte Carlo) 기반 조건부옵션 엔진을 논문과 기존 코드 분석을 통해 단계적으로 구현했습니다. 해당 업무에서 모델의 수학적 정의를 수치화하고 코드로 구현한 뒤 검증하는 전 과정을 수행하며 프라이싱·몬테카를로 시뮬레이션 역량과 Python 구현 능력을 확보했습니다. 촉박한 일정에서는 요구사항을 모듈화하고 우선순위를 정해 핵심 기능을 먼저 구현해 기한 내 결과물을 납품했고, 후속 피드백을 반영해 품질을 개선했습니다. 이 경험을 통해 모델의 가정·한계를 검증하는 습관, 수치적 안정성 확보, 실무용 코드 최적화 능력을 갖추게 되었으며 이는 데스크 환경에서 빠르게 검증 가능한 툴을 만들고 리스크 관리 체계를 설계하는 데 큰 강점이 될 것입니다.\n\n3. Q3.가장 열정적으로 도전했던 경험, 문제의식을 바탕으로 관행을 개선한 경험 등 도전적인 과제를 수행하는 과정에서 성공적인 성과를 만들어 낸 사례를 구체적으로 기술해주세요. (1000자)\nKRX 파생상품 경시대회에서 팀

In [ ]:
def high_level_summary_feedback(prototype_response: str, highlight_point: str = ""):
    question = f"""
아래 초안 원문과 강조할 부분을 참고해서 경험을 고차원적으로 이해하고 자기소개서에 바로 쓸 수 있게 요약해줘

초안 원문
{prototype_response}

강조할 부분
{highlight_point}

규칙
경험은 하나의 경험 단위로만 다뤄 경험을 섞지 마
원문에 없는 사실 수치 조직명 역할을 지어내지 마
이모티콘과 장식용 특수문자 불릿 화살표 같은 표현은 쓰지 마
자기소개서 문장처럼 자연스럽게 단락으로만 써

고차원 유도 규칙
사건 나열 금지 의사결정 중심으로 써
진짜 문제 불확실성과 제약 선택지와 판단 기준 리스크와 트레이드오프를 먼저 정리해
성과는 노력 대신 레버와 지표 변화로 설명해 핵심 변수 1에서 2개와 그 변수를 움직인 방식으로 써
역할은 업무가 아니라 책임으로 재정의해 품질 속도 리스크 조율 비용 중 무엇에 책임졌는지 드러내
역량은 기술 스킬 문제 해결 스킬 원칙 가치 3단으로 내재되게 써
마지막에 재현 가능성 일반화 문장을 넣어 다른 환경에서도 적용 가능한 조건을 제시해
문장 구조는 반드시 아래 순서를 따른다
문제의 본질 1문장
제약과 불확실성 1문장
판단 기준과 선택 1문장
핵심 행동과 레버 2에서 3문장
결과와 검증 1문장
일반화와 재현 조건 1문장

output:
1. (문항 그대로 작성)
(내용)

2. (문항 그대로 작성)
(내용)
"""

    response = client.responses.create(
        model="gpt-5-mini-2025-08-07",
        reasoning={"effort": "medium"},
        input=[
            {
                "role": "system",
                "content": (
                    "너는 자기소개서 작성 전문가다. "
                    "단일 경험을 의사결정과 레버 중심으로 고차원화하여 자기소개서에 바로 쓸 수 있는 문장으로 재구성한다. "
                    "한국어로 자연스럽고 단정한 문장으로 작성한다."
                ),
            },
            {
                "role": "developer",
                "content": (
                    "Hard rules\n"
                    "Treat the input as exactly one experience and do not merge with other experiences\n"
                    "Do not invent any facts metrics or claims not present in the experience text\n"
                    "No emojis and no decorative symbols such as bullets or arrows\n"
                    "Write in natural paragraphs suitable for a self introduction\n"
                    "Follow the output format exactly\n"
                    "In section 1 produce the high level summary using the required sentence order\n"
                    "In section 2 produce brief feedback questions that help the user fill missing decision lever metric tradeoff risk and reproducibility details\n"
                ),
            },
            {"role": "user", "content": question},
        ],
    )

    return response.output_text
